# Answering Business Questions Using SQL

![](https://www.groovehq.com/blog/wp-content/uploads/2017/05/best-of-q-and-a.jpeg)

Image Source: [Groove HQ](https://www.groovehq.com/blog/best-of-q-and-a)

## Table of Contents
---
- [Introduction](#Introduction)
- [Importing Libraries](#Importing-Libraries)
- [Connecting to the Database](#Connecting-to-the-Database)
- [Business Questions](#Business-Questions)
    - [Selecting Albums to Purchase](#A.-Selecting-Albums-to-Purchase)
    - [Analyzing Employee Sales Performance](#B.-Analyzing-Employee-Sales-Performance)
    - [Analyzing Sales by Country](#C.-Analyzing-Sales-by-Country)
    - [Albums vs Individual Tracks](#D.-Albums-vs-Individual-Tracks)
- [Conclusion](#Conclusion)

## Introduction
---
As Business Analysts, we have been contacted by a fictional online music store called Chinook. The company wants our help with answering some business questions.

They provided access to their [database](https://github.com/lerocha/chinook-database), which contains information about the artists, songs, and albums from the music store. Employee information, customer information, and the details of purchase transactions can also be found in the database. The entity relationship diagram below provides a structural overview of the Chinook database:

<img src='https://s3.amazonaws.com/dq-content/189/chinook-schema.svg'
     height= '500'
     width='500'/>
     
## Importing Libraries

In [1]:
import numpy as np
import pandas as pd

## Connecting to the Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Let's start by getting familiar with our data. Remember that we can query the database to get a list of all tables and views in our database:

In [3]:
%%sql                  
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


>- The Chinook database contains 11 tables with information about the store's tracks, employees, customers, and purchase transactions.

## Business Questions

### A. Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We need to write a query and  make a recommendation for the three artists whose albums we should purchase for the store.

In [4]:
%%sql
WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name Genre,
    count(uts.invoice_line_id) "Tracks Sold",
    ROUND(cast(count(uts.invoice_line_id) AS FLOAT) 
          / (SELECT COUNT(*) from usa_tracks_sold) * 100,2)
          "Percentage Sold"
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


Genre,Tracks Sold,Percentage Sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


#### Results
>Based on the genre sales pattern in the USA. Chinook should select these options from the list of available albums.
>- **Punk:** Red Tone
>- **Blues:** Slim Jim Bites 
>- **Pop:** Meteor and the Girls
  
>It's worth noting that these three genres only make up **17%** of total sales. To maximize profitability, the company should be on the lookout for Rock songs since they account for **53%** of sales in the US market.

### B. Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql
WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         COUNT(c.customer_id) AS num_customers,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name Employee,
    e.hire_date "Hire Date",
    num_customers "Number of Customers",
    '$' || ROUND(SUM(csrs.total),2) "Total Sales"
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

Done.


Employee,Hire Date,Number of Customers,Total Sales
Jane Peacock,2017-04-01 00:00:00,8,$1731.51
Margaret Park,2017-05-03 00:00:00,5,$1584.0
Steve Johnson,2017-10-17 00:00:00,13,$1393.92


#### Results
>- Of the three sales employees, Jane achieved the highest total sales of **1,731 dollars**. Margaret Park comes second, and Steve occupies the last place with **1,393 dollars** in sales. This difference in sales is understandable, considering that Jane and Margaret were employed about **five** months before Steve. 

### C. Analyzing Sales by Country

Chinook wants to understand how sales are distributed across different countries. The company intends to identify countries with growth potential and may even run advertising campaigns in these countries. In specific, we would like to calculate data for each country on the:

>- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [9]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales "Total Sales",
    average_order "Average Order",
    customer_lifetime_value "Customer Lifetime Value"
FROM
    (
    SELECT
        Country,
        count(distinct customer_id) Customers,
        '$' || ROUND(SUM(unit_price),2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id),2) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id),2) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY Customers DESC, total_sales DESC
    );

Done.


Country,Customers,Total Sales,Average Order,Customer Lifetime Value
Other,15,$1094.94,7.45,73.0
USA,13,$1040.49,7.94,80.04
Canada,8,$535.59,7.05,66.95
Brazil,5,$427.68,7.01,85.54
France,5,$389.07,7.78,77.81
Germany,4,$334.62,8.16,83.66
United Kingdom,3,$245.52,8.77,81.84
Czech Republic,2,$273.24,9.11,136.62
Portugal,2,$185.13,6.38,92.57
India,2,$183.15,8.72,91.58


#### Results
>- The Majority of Chinook sales arise from **US** and **Canada**. These two countries dominate in customer base as well as sales. However, customers spend lesser per invoice in these countries. 
>- Although the **Czech Republic, Portugal, and India** record fewer customers and lesser sales values,  customers spend more per invoice than in other countries. To increase revenue from these three markets, Chinook could run marketing campaigns to expand its customer base.
   
Since the data from each country are relatively few, the initial marketing campaigns should be small. They should aim to collect and analyze customer data so that the right trends are confidently identified.

### D. Albums vs Individual Tracks

Chinook organizes its store to allow purchases in one of two ways. Customers can purchase a whole album or a collection of one or more individual tracks.

The store does not let customers purchase a whole album and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers buy albums, they are charged the same price as if they had purchased each of those tracks separately.

Chinook is considering changing its purchasing strategy to save money. The company wants to purchase only the most popular tracks from record companies instead of buying every track from an album. We have been asked to determine what percentage of Chinook's sales are individual tracks vs. whole albums. Management can use this data to understand the effect of their decision on overall revenue.

In [7]:
%%sql
-- Collate the list of invoices and their albums
WITH t1 AS(SELECT invoice_id,
                  album_id
             FROM invoice_line
             JOIN track
            USING(track_id)
             LEFT JOIN album
            USING(album_id)
            GROUP BY 1
          ),

-- Check if all tracks in an invoice belong to an album
      t2 AS(SELECT CASE WHEN
                             (SELECT track_id
                                FROM invoice_line il
                               WHERE t1.invoice_id = il.invoice_id
                              EXCEPT
                              SELECT track_id
                                FROM track
                               WHERE track.album_id = t1.album_id
                             ) IS NULL
                        AND
                             (SELECT track_id
                                FROM track
                               WHERE track.album_id = t1.album_id
                              EXCEPT
                              SELECT track_id
                                FROM invoice_line il
                               WHERE t1.invoice_id = il.invoice_id
                             ) IS NULL
                        THEN 'Yes' 
                        ELSE 'No' 
                        END AS album_purchase
               
              FROM t1
          )
    
-- Collate album purchase frequencies          
SELECT album_purchase, 
       COUNT(1) AS num_invoices,
       ROUND(
             CAST(COUNT(1) * 100 AS Float) / (SELECT COUNT(1) FROM t2)
            , 1) AS percentage
  FROM t2
 GROUP BY album_purchase;

Done.


album_purchase,num_invoices,percentage
No,500,81.4
Yes,114,18.6


#### Results
> Most purchases (81%) from the store are individual tracks. However, in about 19% of cases, customers buy entire albums. Chinook should be careful with purchasing only the most popular tracks since it risks losing revenue from customers who purchase entire albums.

## Conclusion

During this project, we helped a fictional company address questions that may help improve profitability. We can advise on aspects of the company's business from various situations presented.

- Regarding the new record label, the company should make the best out of genres with high popularity in the US. They should select Hip-Hop, Punk, and Pop from the current options. Chinook should also be on the lookout for Rock songs since they account for the USA's most significant sales (53%).

- Jane Peacock is Chinook's best-performing salesperson. Steve Johnson appears to be underperforming in total sales, but this is only because he joined four months later than the others.

- US and Canada have the most extensive customer base, but customers spend less per transaction. The Czech Republic, India, and Portugal have the highest spending customers. Chinook could launch a marketing campaign to increase the customer base in the three regions.

- The plan to focus only on popular tracks may sound viable. However, it risks losing revenue from customers who prefer entire albums. It is better to conduct surveys and gather customer perspectives before implementing any new plan.